In [8]:
import gzip
import json

import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
fl=gzip.open('goemotions.json.gz')
res=json.load(fl)
#test
print(res[0])

['That game hurt.', 'sadness', 'negative']


In [10]:
df=pd.DataFrame(res,columns=["post","emotion","sentiment"])
emotions=df.drop(columns=['sentiment','post'])
sentiments=df.drop(columns=['emotion','post'])

In [11]:
posts=df.drop(columns=['emotion','sentiment'])
numpy_posts=posts.to_numpy().flatten()
numpy_emotions=emotions.to_numpy().flatten()
numpy_sentiments=sentiments.to_numpy().flatten()

In [22]:
import gensim.downloader as api
embedding=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
# import gensim
# from gensim import models
# from gensim.models import Word2Vec
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [13]:
print(numpy_posts)

['That game hurt.' "You do right, if you don't care then fuck 'em!"
 'Man I love reddit.' ...
 'Well when you’ve imported about a gazillion of them I or your country it’s gets serious.'
 'That looks amazing'
 "The FDA has plenty to criticize. But like here, it's usually criticized horribly off base. It needs to grow some balls and actually enforce things. "]


In [21]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
token_word = []
for words in numpy_posts:
 token_word.extend(word_tokenize(words))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/safwanahmed/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
len(token_word)

2642128